# Testing OpenSmile Classes

This notebook provides a sequence of steps to generate a dataframe and load the opensmile features for feature selection and testing.

In [1]:
import pandas as pd
import numpy as np
import sys
import os
import json
from tqdm import tqdm
sys.path.append("/home/ubuntu/MultiModalDeepFake")
from packages.LJDataLoader import LJDataLoader
from packages.SmileFeatureLoader import smileFeatureLoader

### Part 1: Creating the dataframe with the train-dev-test split

In [3]:
#load LJ 16khz data
#
file_path = '/home/ubuntu/data/wavefake_data/LJ_metadata_16000KHz.csv'
loader = LJDataLoader(data_path=file_path)

In [4]:
#split data into train, val, test
loader.splitData()


13055
7833 10444


In [6]:
#get the column names
cols = list(loader.metadata.columns[5:15])
cols

['Real',
 'Full_Band_MelGan',
 'HifiGan',
 'MelGan',
 'MelGanLarge',
 'Multi_Band_MelGan',
 'Parallel_WaveGan',
 'Waveglow',
 'ElevenLabs',
 'UberDuck']

In [7]:
#generate the dataframe
df = loader.generateFinalDataFrame(real_col=cols[0], fake_cols=cols[1:], single_id_entry=True)

In [8]:
loader.metadata.shape

(13055, 18)

In [9]:
df.head()

,path,label,multiclass_label,type
0,/home/ubuntu/data/wavefake_data/generated_audi...,1,6,train
1,/home/ubuntu/data/wavefake_data/generated_audi...,1,9,train
2,/home/ubuntu/data/wavefake_data/generated_audi...,1,1,train
3,/home/ubuntu/data/wavefake_data/generated_audi...,1,3,train
4,/home/ubuntu/data/wavefake_data/generated_audi...,1,6,train


__Why are there 130550 data points if `single_id_entry=False`__

In [10]:
df.shape

(13055, 4)

In [11]:
# sample 1000 rows from df for testing
test_df = df.sample(n=1000, random_state=42)

In [12]:
test_df.head()

,path,label,multiclass_label,type
2614,/home/ubuntu/data/wavefake_data/generated_audi...,1,8,train
10003,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,0,0,dev
12056,/home/ubuntu/data/wavefake_data/generated_audi...,1,8,test
2924,/home/ubuntu/data/wavefake_data/generated_audi...,1,8,train
2495,/home/ubuntu/data/wavefake_data/generated_audi...,1,2,train


## 

## Loading the data

Final result should have the following columns with 6373 smile features

metadata=['id', 'path', 'split', 'type', 'label', 'multiclass_label', 'duration(seconds)']

#### Testing Code

In [69]:
test_df.path.apply(lambda x: x.split('/')[-3]).value_counts()

ljspeech_elevenlabs           123
ljspeech_hifiGAN              110
ljspeech_melgan               109
ljspeech_parallel_wavegan     108
ljspeech_uberduck             102
ljspeech_melgan_large         101
ljspeech_multi_band_melgan     96
wavs                           89
ljspeech_full_band_melgan      87
ljspeech_waveglow              75
Name: path, dtype: int64

In [72]:
test_df.path.apply(lambda x: '/'.join(x.split('/')[:-1])).unique()

array(['/home/ubuntu/data/wavefake_data/generated_audio/ljspeech_hifiGAN/16000KHz',
       '/home/ubuntu/data/wavefake_data/generated_audio/ljspeech_melgan_large/16000KHz',
       '/home/ubuntu/data/wavefake_data/generated_audio/ljspeech_melgan/16000KHz',
       '/home/ubuntu/data/wavefake_data/generated_audio/ljspeech_waveglow/16000KHz',
       '/home/ubuntu/data/wavefake_data/generated_audio/ljspeech_uberduck/16000KHz',
       '/home/ubuntu/data/wavefake_data/generated_audio/ljspeech_multi_band_melgan/16000KHz',
       '/home/ubuntu/data/wavefake_data/generated_audio/ljspeech_full_band_melgan/16000KHz',
       '/home/ubuntu/data/wavefake_data/generated_audio/ljspeech_elevenlabs/16000KHz',
       '/home/ubuntu/data/wavefake_data/generated_audio/ljspeech_parallel_wavegan/16000KHz',
       '/home/ubuntu/data/wavefake_data/LJSpeech_1.1/wavs/16000KHz'],
      dtype=object)

In [51]:
test_df.path.iloc[0]

'/home/ubuntu/data/wavefake_data/generated_audio/ljspeech_hifiGAN/16000KHz/LJ035-0150_generated.wav'

In [52]:
'/home/ubuntu/data/OpenSmile/04-2023-OSF-feature-csvs/OSF-2023-04-01-InTheWildFakes-Biden-16000KHz.csv'

'/home/ubuntu/data/OpenSmile/04-2023-OSF-feature-csvs/OSF-2023-04-01-InTheWildFakes-Biden-16000KHz.csv'

In [58]:
#list files in of a particular extension in a directory
def list_files(directory, extension):
    return [f for f in os.listdir(directory) if f.endswith('.' + extension)]

file_list = list_files('/home/ubuntu/data/OpenSmile/04-2023-OSF-feature-csvs', 'csv')
file_list


['OSF-2023-04-01-wavefake_data-LJSpeech_1.1-wavs-Original.csv',
 'OSF-2023-04-01-wavefake_data-generated_audio-ljspeech_hifiGAN-16000KHz.csv',
 'OSF-2023-04-01-InTheWildFakes-Biden-16000KHz.csv',
 'OSF-2023-04-01-wavefake_data-generated_audio-ljspeech_melgan_large-16000KHz.csv',
 'OSF-2023-04-01-wavefake_data-generated_audio-ljspeech_melgan-Original.csv',
 'OSF-2023-04-01-world-leaders-dataset-11LabsDeepFakes-Biden-44Khz.csv',
 'OSF-2023-04-01-wavefake_data-generated_audio-ljspeech_multi_band_melgan-16000KHz.csv',
 'OSF-2023-04-01-wavefake_data-LJSpeech_1.1-wavs-16000KHz.csv',
 'OSF-2023-04-01-wavefake_data-generated_audio-ljspeech_waveglow-Original.csv',
 'OSF-2023-04-06-wavefake_data-generated_audio-ljspeech_uberduck-16000KHz.csv',
 'OSF-2023-04-01-wavefake_data-generated_audio-ljspeech_hifiGAN-Original.csv',
 'OSF-2023-04-01-world-leaders-dataset-WavFiles-biden_wav_audio-44KHz.csv',
 'OSF-2023-04-01-world-leaders-dataset-WavFiles-biden_wav_audio-16KHz.csv',
 'OSF-2023-04-01-wavefake

In [74]:
len(file_list)

26

#### Creating file mappings JSON

In [75]:
file_mapping = dict()

#biden real
file_mapping['/home/ubuntu/data/world-leaders-dataset/WavFiles/biden_wav_audio/44KHz'] = '/home/ubuntu/data/OpenSmile/04-2023-OSF-feature-csvs/OSF-2023-04-01-world-leaders-dataset-WavFiles-biden_wav_audio-44KHz.csv'
file_mapping['/home/ubuntu/data/world-leaders-dataset/WavFiles/biden_wav_audio/16KHz'] = '/home/ubuntu/data/OpenSmile/04-2023-OSF-feature-csvs/OSF-2023-04-01-world-leaders-dataset-WavFiles-biden_wav_audio-16KHz.csv'

#biden eleven labs
file_mapping['/home/ubuntu/data/world-leaders-dataset/11LabsDeepFakes/Biden/44Khz'] = '/home/ubuntu/data/OpenSmile/04-2023-OSF-feature-csvs/OSF-2023-04-01-world-leaders-dataset-11LabsDeepFakes-Biden-44Khz.csv'
file_mapping['/home/ubuntu/data/world-leaders-dataset/11LabsDeepFakes/Biden/16Khz'] = '/home/ubuntu/data/OpenSmile/04-2023-OSF-feature-csvs/OSF-2023-04-01-world-leaders-dataset-11LabsDeepFakes-Biden-16Khz.csv'

#biden in the wild
file_mapping['/home/ubuntu/data/InTheWildFakes/Biden/Original'] = '/home/ubuntu/data/OpenSmile/04-2023-OSF-feature-csvs/OSF-2023-04-01-InTheWildFakes-Biden-Original.csv'
file_mapping['/home/ubuntu/data/InTheWildFakes/Biden/16000KHz'] = '/home/ubuntu/data/OpenSmile/04-2023-OSF-feature-csvs/OSF-2023-04-01-InTheWildFakes-Biden-16000KHz.csv'

#ljspeech real
file_mapping['/home/ubuntu/data/wavefake_data/LJSpeech_1.1/wavs/Original'] = '/home/ubuntu/data/OpenSmile/04-2023-OSF-feature-csvs/OSF-2023-04-01-wavefake_data-LJSpeech_1.1-wavs-Original.csv'
file_mapping['/home/ubuntu/data/wavefake_data/LJSpeech_1.1/wavs/16000KHz'] = '/home/ubuntu/data/OpenSmile/04-2023-OSF-feature-csvs/OSF-2023-04-01-wavefake_data-LJSpeech_1.1-wavs-16000KHz.csv'

#ljspeech eleven labs
file_mapping['/home/ubuntu/data/wavefake_data/generated_audio/ljspeech_elevenlabs/Original'] = '/home/ubuntu/data/OpenSmile/04-2023-OSF-feature-csvs/OSF-2023-04-01-wavefake_data-generated_audio-ljspeech_elevenlabs-Original.csv'
file_mapping['/home/ubuntu/data/wavefake_data/generated_audio/ljspeech_elevenlabs/16000KHz'] = '/home/ubuntu/data/OpenSmile/04-2023-OSF-feature-csvs/OSF-2023-04-01-wavefake_data-generated_audio-ljspeech_elevenlabs-16000KHz.csv'

#ljspeech uberduck
file_mapping['/home/ubuntu/data/wavefake_data/generated_audio/ljspeech_uberduck/Original'] = '/home/ubuntu/data/OpenSmile/04-2023-OSF-feature-csvs/OSF-2023-04-06-wavefake_data-generated_audio-ljspeech_uberduck-Original.csv'
file_mapping['/home/ubuntu/data/wavefake_data/generated_audio/ljspeech_uberduck/16000KHz'] = '/home/ubuntu/data/OpenSmile/04-2023-OSF-feature-csvs/OSF-2023-04-06-wavefake_data-generated_audio-ljspeech_uberduck-16000KHz.csv'

#ljspeech melgan
file_mapping['/home/ubuntu/data/wavefake_data/generated_audio/ljspeech_melgan/Original'] = '/home/ubuntu/data/OpenSmile/04-2023-OSF-feature-csvs/OSF-2023-04-01-wavefake_data-generated_audio-ljspeech_melgan-Original.csv'
file_mapping['/home/ubuntu/data/wavefake_data/generated_audio/ljspeech_melgan/16000KHz'] = '/home/ubuntu/data/OpenSmile/04-2023-OSF-feature-csvs/OSF-2023-04-01-wavefake_data-generated_audio-ljspeech_melgan-16000KHz.csv'

#ljspeech melgan large
file_mapping['/home/ubuntu/data/wavefake_data/generated_audio/ljspeech_melgan_large/Original'] = '/home/ubuntu/data/OpenSmile/04-2023-OSF-feature-csvs/OSF-2023-04-01-wavefake_data-generated_audio-ljspeech_melgan_large-Original.csv'
file_mapping['/home/ubuntu/data/wavefake_data/generated_audio/ljspeech_melgan_large/16000KHz'] = '/home/ubuntu/data/OpenSmile/04-2023-OSF-feature-csvs/OSF-2023-04-01-wavefake_data-generated_audio-ljspeech_melgan_large-16000KHz.csv'

#ljspeech multi band melgan
file_mapping['/home/ubuntu/data/wavefake_data/generated_audio/ljspeech_multi_band_melgan/Original'] = '/home/ubuntu/data/OpenSmile/04-2023-OSF-feature-csvs/OSF-2023-04-01-wavefake_data-generated_audio-ljspeech_multi_band_melgan-Original.csv'
file_mapping['/home/ubuntu/data/wavefake_data/generated_audio/ljspeech_multi_band_melgan/16000KHz'] = '/home/ubuntu/data/OpenSmile/04-2023-OSF-feature-csvs/OSF-2023-04-01-wavefake_data-generated_audio-ljspeech_multi_band_melgan-16000KHz.csv'

#ljspeech full band melgan
file_mapping['/home/ubuntu/data/wavefake_data/generated_audio/ljspeech_full_band_melgan/Original'] = '/home/ubuntu/data/OpenSmile/04-2023-OSF-feature-csvs/OSF-2023-04-01-wavefake_data-generated_audio-ljspeech_full_band_melgan-Original.csv'
file_mapping['/home/ubuntu/data/wavefake_data/generated_audio/ljspeech_full_band_melgan/16000KHz'] = '/home/ubuntu/data/OpenSmile/04-2023-OSF-feature-csvs/OSF-2023-04-01-wavefake_data-generated_audio-ljspeech_full_band_melgan-16000KHz.csv'

#ljspeech hifiGAN
file_mapping['/home/ubuntu/data/wavefake_data/generated_audio/ljspeech_hifiGAN/Original'] = '/home/ubuntu/data/OpenSmile/04-2023-OSF-feature-csvs/OSF-2023-04-01-wavefake_data-generated_audio-ljspeech_hifiGAN-Original.csv'
file_mapping['/home/ubuntu/data/wavefake_data/generated_audio/ljspeech_hifiGAN/16000KHz'] = '/home/ubuntu/data/OpenSmile/04-2023-OSF-feature-csvs/OSF-2023-04-01-wavefake_data-generated_audio-ljspeech_hifiGAN-16000KHz.csv'

#ljspeech parallel wavegan
file_mapping['/home/ubuntu/data/wavefake_data/generated_audio/ljspeech_parallel_wavegan/Original'] = '/home/ubuntu/data/OpenSmile/04-2023-OSF-feature-csvs/OSF-2023-04-01-wavefake_data-generated_audio-ljspeech_parallel_wavegan-Original.csv'
file_mapping['/home/ubuntu/data/wavefake_data/generated_audio/ljspeech_parallel_wavegan/16000KHz'] = '/home/ubuntu/data/OpenSmile/04-2023-OSF-feature-csvs/OSF-2023-04-01-wavefake_data-generated_audio-ljspeech_parallel_wavegan-16000KHz.csv'

#ljspeech waveglow
file_mapping['/home/ubuntu/data/wavefake_data/generated_audio/ljspeech_waveglow/Original'] = '/home/ubuntu/data/OpenSmile/04-2023-OSF-feature-csvs/OSF-2023-04-01-wavefake_data-generated_audio-ljspeech_waveglow-Original.csv'
file_mapping['/home/ubuntu/data/wavefake_data/generated_audio/ljspeech_waveglow/16000KHz'] = '/home/ubuntu/data/OpenSmile/04-2023-OSF-feature-csvs/OSF-2023-04-01-wavefake_data-generated_audio-ljspeech_waveglow-16000KHz.csv'

file_mapping

{'/home/ubuntu/data/world-leaders-dataset/WavFiles/biden_wav_audio/44KHz': '/home/ubuntu/data/OpenSmile/04-2023-OSF-feature-csvs/OSF-2023-04-01-world-leaders-dataset-WavFiles-biden_wav_audio-44KHz.csv',
 '/home/ubuntu/data/world-leaders-dataset/WavFiles/biden_wav_audio/16KHz': '/home/ubuntu/data/OpenSmile/04-2023-OSF-feature-csvs/OSF-2023-04-01-world-leaders-dataset-WavFiles-biden_wav_audio-16KHz.csv',
 '/home/ubuntu/data/world-leaders-dataset/11LabsDeepFakes/Biden/44Khz': '/home/ubuntu/data/OpenSmile/04-2023-OSF-feature-csvs/OSF-2023-04-01-world-leaders-dataset-11LabsDeepFakes-Biden-44Khz.csv',
 '/home/ubuntu/data/world-leaders-dataset/11LabsDeepFakes/Biden/16Khz': '/home/ubuntu/data/OpenSmile/04-2023-OSF-feature-csvs/OSF-2023-04-01-world-leaders-dataset-11LabsDeepFakes-Biden-16Khz.csv',
 '/home/ubuntu/data/InTheWildFakes/Biden/Original': '/home/ubuntu/data/OpenSmile/04-2023-OSF-feature-csvs/OSF-2023-04-01-InTheWildFakes-Biden-Original.csv',
 '/home/ubuntu/data/InTheWildFakes/Biden/16

In [77]:
import json

file_mappings = '/home/ubuntu/data/OpenSmile/04-2023-OSF-feature-csvs/' + 'file_mappings.json' 

with open(file_mappings, "w") as json_file:
    json.dump(file_mapping, json_file, indent=4)

## Scratchpad

In [78]:
test_df_2 = pd.read_csv('/home/ubuntu/data/OpenSmile/04-2023-OSF-feature-csvs/OSF-2023-04-06-wavefake_data-generated_audio-ljspeech_uberduck-16000KHz.csv')

In [79]:
test_df_2.shape

(13094, 6375)

In [80]:
test_df.head()

,path,label,multiclass_label,type
2614,/home/ubuntu/data/wavefake_data/generated_audi...,1,2,train
10003,/home/ubuntu/data/wavefake_data/generated_audi...,1,4,dev
12056,/home/ubuntu/data/wavefake_data/generated_audi...,1,3,test
2924,/home/ubuntu/data/wavefake_data/generated_audi...,1,4,train
2495,/home/ubuntu/data/wavefake_data/generated_audi...,1,3,train


In [81]:
file_mappings

'/home/ubuntu/data/OpenSmile/04-2023-OSF-feature-csvs/file_mappings.json'

In [20]:
class smileFeatureLoader:
    def __init__(self, df, 
                 file_mappings= '/home/ubuntu/data/OpenSmile/04-2023-OSF-feature-csvs/file_mappings.json') -> None:

        self.df = df
        self.df['path_keys'] = df.path.apply(lambda x: '/'.join(x.split('/')[:-1]))
        
        with open(file_mappings) as f:
            self.file_mappings = json.load(f)
        
        self.merged_df = self._load_smile_features()
        
    def _load_smile_features(self):

        tqdm.pandas()

        merged_df = pd.DataFrame()
        
        #iterate through all the unique path keys in the provided df
        for path_key in tqdm(self.df.path_keys.unique()):
            
            #load the precomputed smile features for files in the path key
            csv_path = self.file_mappings[path_key]
            right_df = pd.read_csv(csv_path)
            
            #create the left df by filtering the provided df by the path key
            left_df = self.df[self.df.path_keys == path_key]

            #merge the left and right dfs
            merged_df = pd.concat([merged_df, pd.merge(left_df, right_df, left_on='path', right_on='file', how='left')], axis=0)

        return merged_df
                
                

In [21]:
sm = smileFeatureLoader(test_df)

14it [00:12,  1.15it/s]00:00<?, ?it/s]
14it [00:12,  1.14it/s]00:12<01:51, 12.43s/it]
4it [00:04,  1.09s/it][00:24<01:39, 12.48s/it]
 20%|██        | 2/10 [00:29<01:57, 14.66s/it]


ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [13]:
sm.merged_df

,path,label,multiclass_label,type,path_keys,file,duration(seconds),audspec_lengthL1norm_sma_range,audspec_lengthL1norm_sma_maxPos,audspec_lengthL1norm_sma_minPos,...,mfcc_sma_de[14]_peakRangeAbs,mfcc_sma_de[14]_peakRangeRel,mfcc_sma_de[14]_peakMeanAbs,mfcc_sma_de[14]_peakMeanMeanDist,mfcc_sma_de[14]_peakMeanRel,mfcc_sma_de[14]_minRangeRel,mfcc_sma_de[14]_meanRisingSlope,mfcc_sma_de[14]_stddevRisingSlope,mfcc_sma_de[14]_meanFallingSlope,mfcc_sma_de[14]_stddevFallingSlope
0,/home/ubuntu/data/wavefake_data/generated_audi...,1,8,train,/home/ubuntu/data/wavefake_data/generated_audi...,/home/ubuntu/data/wavefake_data/generated_audi...,6.269,1.833148,0.583199,0.000000,...,10.114010,0.511215,3.687236,3.687813,-20.000000,0.605559,140.96674,61.022186,139.871150,83.242775
1,/home/ubuntu/data/wavefake_data/generated_audi...,1,8,train,/home/ubuntu/data/wavefake_data/generated_audi...,/home/ubuntu/data/wavefake_data/generated_audi...,2.089,2.192309,0.144279,0.000000,...,5.299999,0.410885,2.593824,2.601382,-19.980114,0.613261,130.84023,61.562347,104.060470,43.534836
2,/home/ubuntu/data/wavefake_data/generated_audi...,1,8,train,/home/ubuntu/data/wavefake_data/generated_audi...,/home/ubuntu/data/wavefake_data/generated_audi...,9.012,1.809207,0.024608,0.000000,...,10.790438,0.525003,3.622012,3.623859,-20.000000,0.531072,147.01364,78.597780,140.556670,68.385190
3,/home/ubuntu/data/wavefake_data/generated_audi...,1,8,train,/home/ubuntu/data/wavefake_data/generated_audi...,/home/ubuntu/data/wavefake_data/generated_audi...,6.530,1.916828,0.010836,0.000000,...,10.668040,0.543688,3.270228,3.269604,20.000000,0.532827,148.19579,77.316980,137.104170,65.971275
4,/home/ubuntu/data/wavefake_data/generated_audi...,1,8,train,/home/ubuntu/data/wavefake_data/generated_audi...,/home/ubuntu/data/wavefake_data/generated_audi...,3.761,1.712983,0.243902,0.000000,...,6.755691,0.469319,2.834880,2.844430,-19.966377,0.649432,126.90704,65.421590,130.091920,60.061220
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,/home/ubuntu/data/wavefake_data/generated_audi...,1,5,train,/home/ubuntu/data/wavefake_data/generated_audi...,/home/ubuntu/data/wavefake_data/generated_audi...,8.661,1.553160,0.076834,0.649593,...,10.735183,0.646978,2.648336,2.641622,19.988420,0.456448,120.63265,66.592440,117.374954,65.537590
96,/home/ubuntu/data/wavefake_data/generated_audi...,1,5,test,/home/ubuntu/data/wavefake_data/generated_audi...,/home/ubuntu/data/wavefake_data/generated_audi...,8.336,1.772172,0.700968,0.648910,...,7.625343,0.608741,2.418311,2.424134,-19.990600,0.610165,104.94406,54.201775,115.714910,57.590630
97,/home/ubuntu/data/wavefake_data/generated_audi...,1,5,train,/home/ubuntu/data/wavefake_data/generated_audi...,/home/ubuntu/data/wavefake_data/generated_audi...,6.490,1.954121,0.951713,0.476636,...,8.423761,0.647947,2.706032,2.693978,19.916290,0.429277,113.65552,59.076927,117.757900,53.465050
98,/home/ubuntu/data/wavefake_data/generated_audi...,1,5,test,/home/ubuntu/data/wavefake_data/generated_audi...,/home/ubuntu/data/wavefake_data/generated_audi...,5.932,1.838428,0.206485,0.186007,...,7.011283,0.556861,2.458248,2.465018,-19.983952,0.514018,111.73879,49.925026,105.887634,54.165188
